<a href="https://colab.research.google.com/github/ErangaOttachchige/Final-Year-Research-Project/blob/main/02_stage1_detection_filter_training_OPTIMIZED_for_Colab_Free_T4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stage 1 Binary Filter:

Label: animal vs non_animal (where non_animal = empty + car)

Fast: uses your tensor cache (no local copy, no Drive rsync)

Stable: low RAM, no crashes

Saves to Drive: model + label mapping

Progress bars: train/val/test

In [4]:
# Mount Drive + paths
from google.colab import drive
drive.mount("/content/drive")

import os
DRIVE_CCT = "/content/drive/MyDrive/datasets/cct20"
PROC_DIR  = f"{DRIVE_CCT}/processed"
CSV_STAGE1 = f"{PROC_DIR}/cct20_stage1_imagelevel.csv"

print("✓ PROC_DIR files:", os.listdir(PROC_DIR))
print("✓ Stage1 CSV exists:", os.path.exists(CSV_STAGE1))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✓ PROC_DIR files: ['cct20_species_annotations.csv', 'cct20_stage1_imagelevel.csv', 'cct20_stage2_species_imagelevel.csv', 'stage2_best_species_efficientnet_b0_optimized.pt', 'stage2_label_mapping.json', 'stage1_best_efficientnet_b0.pt', 'stage1_label_mapping.json', 'stage1_best_binary_efficientnet_b0.pt', 'stage1_binary_label_mapping.json']
✓ Stage1 CSV exists: True


In [5]:
# Install packages + check GPU
!pip -q install timm torchmetrics pandas numpy scikit-learn pillow tqdm

import torch
print("✓ CUDA:", torch.cuda.is_available())
print("✓ GPU :", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


✓ CUDA: True
✓ GPU : Tesla T4


In [6]:
# Load Stage1 CSV + create binary labels
import pandas as pd
import os

df = pd.read_csv(CSV_STAGE1)

# Binary label:
# animal stays animal
# car + empty become non_animal
df["label_bin"] = df["label_stage1"].apply(lambda x: "animal" if x=="animal" else "non_animal")

missing = (~df["path"].apply(os.path.exists)).sum()
print(f"✓ Rows: {len(df)}, Missing paths: {missing}")
print("\nSplit counts:\n", df["split"].value_counts())
print("\nBinary label counts:\n", df["label_bin"].value_counts())


✓ Rows: 57864, Missing paths: 0

Split counts:
 split
test_trans    23275
test_cis      15827
train         13553
val_cis        3484
val_trans      1725
Name: count, dtype: int64

Binary label counts:
 label_bin
animal        51237
non_animal     6627
Name: count, dtype: int64


In [13]:
# ============================================================================
# FIX: Re-stratify splits to ensure empty/car/animal in ALL splits
# ============================================================================
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

print("="*60)
print("FIXING DATA SPLIT - Ensuring balanced animal/car/empty")
print("="*60)

# Check current distribution by original label
print("\nCURRENT train distribution by 3-class label:")
current_train = df[df["split"] == "train"]
print(current_train["label_stage1"].value_counts())

# Check if empty is severely underrepresented
empty_count = (current_train["label_stage1"] == "empty").sum()
car_count = (current_train["label_stage1"] == "car").sum()
animal_count = (current_train["label_stage1"] == "animal").sum()

print(f"\nTrain counts: animal={animal_count}, car={car_count}, empty={empty_count}")

# If empty is < 5% of non_animal OR < 50 samples total, re-stratify
if empty_count < 50 or empty_count < (car_count * 0.05):
    print("\n⚠️ Empty severely underrepresented! Re-stratifying splits...\n")

    # Separate by split type (keep test sets as-is, only re-split train/val)
    test_cis = df[df["split"] == "test_cis"].copy()
    test_trans = df[df["split"] == "test_trans"].copy()

    # Pool together current train + val for re-splitting
    trainable = df[df["split"].isin(["train", "val_cis", "val_trans"])].copy()

    print(f"Pooling {len(trainable)} samples for re-split (keeping {len(test_cis)+len(test_trans)} test samples)")
    print("\nPooled distribution by label:")
    print(trainable["label_stage1"].value_counts())

    # Stratified split: 70% train, 15% val_cis, 15% val_trans
    # First split: 70% train, 30% val
    train_idx, val_idx = train_test_split(
        trainable.index,
        test_size=0.30,
        stratify=trainable["label_stage1"],
        random_state=42
    )

    # Second split: split val into val_cis (50%) and val_trans (50%)
    val_subset = trainable.loc[val_idx]
    val_cis_idx, val_trans_idx = train_test_split(
        val_subset.index,
        test_size=0.50,
        stratify=val_subset["label_stage1"],
        random_state=42
    )

    # Assign new splits
    df.loc[train_idx, "split"] = "train"
    df.loc[val_cis_idx, "split"] = "val_cis"
    df.loc[val_trans_idx, "split"] = "val_trans"

    # Verify new distribution
    print("\n✓ NEW train distribution:")
    new_train = df[df["split"] == "train"]
    print(new_train["label_stage1"].value_counts())

    print("\n✓ NEW val_cis distribution:")
    print(df[df["split"] == "val_cis"]["label_stage1"].value_counts())

    print("\n✓ NEW val_trans distribution:")
    print(df[df["split"] == "val_trans"]["label_stage1"].value_counts())

    # Save fixed CSV back to Drive
    fixed_csv = f"{PROC_DIR}/cct20_stage1_imagelevel_FIXED.csv"
    df.to_csv(fixed_csv, index=False)
    print(f"\n💾 Saved fixed CSV to: {fixed_csv}")
    print("   (Original CSV unchanged)")

else:
    print(f"\n✓ Split is OK - empty count ({empty_count}) is sufficient")

print("\n" + "="*60)
print("FINAL SPLIT COUNTS:")
print("="*60)
print(df["split"].value_counts())
print("\nBinary labels:")
df["label_bin"] = df["label_stage1"].apply(lambda x: "animal" if x=="animal" else "non_animal")
print(df["label_bin"].value_counts())

FIXING DATA SPLIT - Ensuring balanced animal/car/empty

CURRENT train distribution by 3-class label:
label_stage1
animal    11898
empty       739
car         496
Name: count, dtype: int64

Train counts: animal=11898, car=496, empty=739

✓ Split is OK - empty count (739) is sufficient

FINAL SPLIT COUNTS:
split
test_trans    23275
test_cis      15827
train         13133
val_trans      2815
val_cis        2814
Name: count, dtype: int64

Binary labels:
label_bin
animal        51237
non_animal     6627
Name: count, dtype: int64


In [18]:
# Ultra-fast caching with optimizations
import glob, hashlib
from PIL import Image
import torchvision.transforms as T
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import torch
import os

CACHE_DIR = "/content/stage1_bin_cache_224"
os.makedirs(CACHE_DIR, exist_ok=True)

def cache_path(img_path):
    return os.path.join(CACHE_DIR, hashlib.md5(img_path.encode()).hexdigest() + ".pt")

df["cache_path"] = df["path"].apply(cache_path)

cached_count = len(glob.glob(CACHE_DIR + "/*.pt"))
print(f"✓ Cached tensors: {cached_count} / {len(df)}")

# Pre-compile transform (slight speed boost)
pre_tf = T.Compose([
    T.Resize((224, 224), antialias=True),  # antialias=True for quality
    T.ToTensor(),
])

def process_one(row):
    """Optimized processing with early exit"""
    cp = row["cache_path"]
    if os.path.exists(cp):
        return
    try:
        # Use Image.open in a context manager (releases file handle faster)
        with Image.open(row["path"]) as img:
            img_rgb = img.convert("RGB")
            x = pre_tf(img_rgb)
        torch.save(x, cp)
    except Exception:
        pass  # Silently skip corrupted images

if cached_count < len(df) * 0.95:
    print("🔄 Caching tensors (8 workers)...")

    # Filter out already-cached rows BEFORE threading (faster)
    rows_to_process = [row for _, row in df.iterrows() if not os.path.exists(row["cache_path"])]

    print(f"   Processing {len(rows_to_process)} new images...")

    with ThreadPoolExecutor(max_workers=8) as ex:
        list(tqdm(
            ex.map(process_one, rows_to_process),
            total=len(rows_to_process),
            desc="Caching Stage1(bin)"
        ))

    print("✓ Cache done:", len(glob.glob(CACHE_DIR + "/*.pt")))
else:
    print("✓ Cache already exists ✅")

# Keep only rows that have cache file (safety)
df["has_cache"] = df["cache_path"].apply(os.path.exists)
df = df[df["has_cache"]].reset_index(drop=True)
print("✓ Usable rows:", len(df))

✓ Cached tensors: 57864 / 57864
✓ Cache already exists ✅
✓ Usable rows: 57864


In [19]:
# Dataset + DataLoaders (balanced sampling + stable RAM)
import numpy as np
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.metrics import f1_score, accuracy_score
import torchvision.transforms as T
from tqdm import tqdm
import torch

# label mapping (binary)
classes = ["animal", "non_animal"]
class_to_idx = {c:i for i,c in enumerate(classes)}
idx_to_class = {i:c for c,i in class_to_idx.items()}
df["y"] = df["label_bin"].map(class_to_idx)

# splits
train_df = df[df["split"]=="train"].reset_index(drop=True)
val_df   = df[df["split"]=="val_cis"].reset_index(drop=True)
valT_df  = df[df["split"]=="val_trans"].reset_index(drop=True)
test_cis_df   = df[df["split"]=="test_cis"].reset_index(drop=True)
test_trans_df = df[df["split"]=="test_trans"].reset_index(drop=True)

print("train:", len(train_df), "val_cis:", len(val_df), "val_trans:", len(valT_df),
      "test_cis:", len(test_cis_df), "test_trans:", len(test_trans_df))
print("\nTrain label counts:\n", train_df["label_bin"].value_counts())

# augmentation applied on tensor
aug_tf = T.Compose([
    T.RandomHorizontalFlip(0.5),
    T.ColorJitter(0.2, 0.2, 0.1),
])

class CachedDS(Dataset):
    def __init__(self, frame, augment=False):
        self.df = frame.reset_index(drop=True)
        self.augment = augment
    def __len__(self):
        return len(self.df)
    def __getitem__(self, i):
        r = self.df.iloc[i]
        x = torch.load(r["cache_path"])
        if self.augment:
            x = aug_tf(x)
        y = int(r["y"])
        return x, y

train_ds = CachedDS(train_df, augment=True)
val_ds   = CachedDS(val_df, augment=False)
valT_ds  = CachedDS(valT_df, augment=False)
test_cis_ds   = CachedDS(test_cis_df, augment=False)
test_trans_ds = CachedDS(test_trans_df, augment=False)

# balanced sampler
counts = train_df["y"].value_counts().sort_index()
w_class = 1.0 / counts
w_sample = train_df["y"].map(w_class).values
sampler = WeightedRandomSampler(torch.tensor(w_sample, dtype=torch.double),
                                num_samples=len(w_sample),
                                replacement=True)

# FIXED: class weights for loss (ensure it matches number of classes)
class_weight = torch.ones(len(classes), dtype=torch.float32)  # Initialize for all classes

for class_idx in counts.index:
    class_weight[class_idx] = counts.sum() / (len(classes) * counts[class_idx])

print("\nCounts:", counts.to_dict())
print("Loss weights (shape={}):", class_weight.shape, class_weight)

# loaders
BATCH_TRAIN = 32
BATCH_EVAL  = 64
NUM_WORKERS = 2

train_loader = DataLoader(train_ds, batch_size=BATCH_TRAIN, sampler=sampler,
                          num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds, batch_size=BATCH_EVAL, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
valT_loader  = DataLoader(valT_ds, batch_size=BATCH_EVAL, shuffle=False,
                          num_workers=NUM_WORKERS, pin_memory=True)
test_cis_loader = DataLoader(test_cis_ds, batch_size=BATCH_EVAL, shuffle=False,
                             num_workers=NUM_WORKERS, pin_memory=True)
test_trans_loader = DataLoader(test_trans_ds, batch_size=BATCH_EVAL, shuffle=False,
                               num_workers=NUM_WORKERS, pin_memory=True)


train: 13133 val_cis: 2814 val_trans: 2815 test_cis: 15827 test_trans: 23275

Train label counts:
 label_bin
animal        11898
non_animal     1235
Name: count, dtype: int64

Counts: {0: 11898, 1: 1235}
Loss weights (shape={}): torch.Size([2]) tensor([0.5519, 5.3170])


In [20]:
# Train EfficientNet-B0 (binary) + save best to Drive
import timm
import torch.nn as nn
from sklearn.metrics import f1_score, accuracy_score
from tqdm import tqdm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print("✓ device:", device)

model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=len(classes)).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weight.to(device))
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4)

scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))
ACCUM_STEPS = 2

def eval_loader(loader, name="eval"):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in tqdm(loader, desc=name, leave=False):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=(device=="cuda")):
                logits = model(x)
            p = logits.argmax(1)
            ys.extend(y.cpu().tolist())
            ps.extend(p.cpu().tolist())
    acc = accuracy_score(ys, ps)
    mf1 = f1_score(ys, ps, average="macro")
    return acc, mf1

SAVE_PATH = f"{PROC_DIR}/stage1_best_binary_efficientnet_b0.pt"
best = -1.0
EPOCHS = 5

for ep in range(1, EPOCHS+1):
    model.train()
    optimizer.zero_grad(set_to_none=True)
    running = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {ep}/{EPOCHS} [train]")
    for i, (x, y) in enumerate(pbar, 1):
        x = x.to(device, non_blocking=True)
        y = y.to(device, non_blocking=True)

        with torch.cuda.amp.autocast(enabled=(device=="cuda")):
            logits = model(x)
            loss = criterion(logits, y) / ACCUM_STEPS

        scaler.scale(loss).backward()

        if i % ACCUM_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)

        running += loss.item() * x.size(0) * ACCUM_STEPS
        pbar.set_postfix({"loss": f"{loss.item()*ACCUM_STEPS:.4f}"})

    train_loss = running / len(train_df)

    val_acc, val_mf1 = eval_loader(val_loader, "val_cis")
    print(f"\nEpoch {ep}: train_loss={train_loss:.4f} | val_cis acc={val_acc:.3f} macroF1={val_mf1:.3f}")

    if len(valT_df) > 0:
        vt_acc, vt_mf1 = eval_loader(valT_loader, "val_trans")
        print(f"          val_trans acc={vt_acc:.3f} macroF1={vt_mf1:.3f}")

    if val_mf1 > best:
        best = val_mf1
        torch.save(model.state_dict(), SAVE_PATH)
        print(f"💾 SAVED BEST → {SAVE_PATH} (macroF1={best:.3f})")


✓ device: cuda


/tmp/ipython-input-4002588860.py:15: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(device=="cuda"))
Epoch 1/5 [train]:   0%|          | 0/411 [00:00<?, ?it/s]/tmp/ipython-input-4002588860.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
val_cis:   0%|          | 0/44 [00:00<?, ?it/s]/tmp/ipython-input-4002588860.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):



Epoch 1: train_loss=0.3297 | val_cis acc=0.855 macroF1=0.738


val_trans:   0%|          | 0/44 [00:00<?, ?it/s]/tmp/ipython-input-4002588860.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


          val_trans acc=0.837 macroF1=0.715
💾 SAVED BEST → /content/drive/MyDrive/datasets/cct20/processed/stage1_best_binary_efficientnet_b0.pt (macroF1=0.738)


...

          val_trans acc=0.921 macroF1=0.825
💾 SAVED BEST → /content/drive/MyDrive/datasets/cct20/processed/stage1_best_binary_efficientnet_b0.pt (macroF1=0.834)


Epoch 5/5 [train]:   0%|          | 0/411 [00:00<?, ?it/s]/tmp/ipython-input-4002588860.py:48: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
val_cis:   0%|          | 0/44 [00:00<?, ?it/s]/tmp/ipython-input-4002588860.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):



Epoch 5: train_loss=0.0320 | val_cis acc=0.938 macroF1=0.854


val_trans:   0%|          | 0/44 [00:00<?, ?it/s]/tmp/ipython-input-4002588860.py:25: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):


          val_trans acc=0.930 macroF1=0.838
💾 SAVED BEST → /content/drive/MyDrive/datasets/cct20/processed/stage1_best_binary_efficientnet_b0.pt (macroF1=0.854)


In [21]:

# Final test evaluation (CIS vs TRANS)
from sklearn.metrics import classification_report
import torch

model.load_state_dict(torch.load(SAVE_PATH, map_location=device))

def full_eval(loader, name="eval"):
    model.eval()
    ys, ps = [], []
    with torch.no_grad():
        for x, y in tqdm(loader, desc=name):
            x = x.to(device, non_blocking=True)
            y = y.to(device, non_blocking=True)
            with torch.cuda.amp.autocast(enabled=(device=="cuda")):
                logits = model(x)
            p = logits.argmax(1)
            ys.extend(y.cpu().tolist())
            ps.extend(p.cpu().tolist())
    return ys, ps

cis_y, cis_p = full_eval(test_cis_loader, "test_cis")
tr_y,  tr_p  = full_eval(test_trans_loader, "test_trans")

print("\n--- CIS REPORT ---")
print(classification_report(cis_y, cis_p, target_names=classes))

print("\n--- TRANS REPORT ---")
print(classification_report(tr_y, tr_p, target_names=classes))


test_cis:   0%|          | 0/248 [00:00<?, ?it/s]/tmp/ipython-input-2556348763.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=(device=="cuda")):
test_trans: 100%|██████████| 364/364 [01:46<00:00,  3.41it/s]


--- CIS REPORT ---
              precision    recall  f1-score   support

      animal       0.96      0.89      0.93     13856
  non_animal       0.50      0.77      0.61      1971

    accuracy                           0.88     15827
   macro avg       0.73      0.83      0.77     15827
weighted avg       0.91      0.88      0.89     15827


--- TRANS REPORT ---
              precision    recall  f1-score   support

      animal       0.95      0.88      0.91     20384
  non_animal       0.45      0.67      0.53      2891

    accuracy                           0.86     23275
   macro avg       0.70      0.78      0.72     23275
weighted avg       0.89      0.86      0.87     23275



In [23]:
# Save label mapping (Drive)
import json

mapping = {"classes": classes, "class_to_idx": class_to_idx}
out_json = f"{PROC_DIR}/stage1_binary_label_mapping.json"
with open(out_json, "w") as f:
    json.dump(mapping, f, indent=2)

print("✓ Saved:", out_json)
print("✓ Saved model:", SAVE_PATH)


✓ Saved: /content/drive/MyDrive/datasets/cct20/processed/stage1_binary_label_mapping.json
✓ Saved model: /content/drive/MyDrive/datasets/cct20/processed/stage1_best_binary_efficientnet_b0.pt
